Did you know you can build C or C++ code and use it in python? You can, and one way to do it is using <a href='https://github.com/swig/swig' target='_blank'>Swig</a> which connect C and C++ programs with many other programming languages.

This notebook wants explaining briefly how to create a straightforward python module using some functions written in C. This is a personal experiment for not forgetting how to do it, based on great <a href='https://www.swig.org/Doc1.3/Python.html#Python_nn4' target='_blank'>documentation of Swing</a>. Adding a simple comparative in execution time using a built magic command. The example is the same presented by Swig documentation as initial example, factorial function, and I'll be focused on this function because is a function who has pwerformance challenges when the parameter value is high.

I'll begin pointing out that for building this basic example we need three files. I mean, when you want coding in C programming language is a good practice begin with header and body **-.h and .c files-** respectively. If you don't know what is each one of these kind of files, I'd suggest you refering to <a href='https://www.amazon.com/-/es/Stephen-Prata/dp/0321776402' target='_blank'> C++ Primer Plus book, by Stephen Prata -chapter 2-</a> or <a href='https://www.amazon.com/Modern-Absolute-Beginners-Introduction-Programming/dp/1484260465' target='_blank'>Modernn C++ for absolute beginners -Chapter 31-</a> to understand better the importance at least of this two C programming language files. The last one is a file with extension **.i**.

Worth it to recall a factorial function is a recursive function, where to get the value of it for a number $x$ you'll need the factorial of $x - 1$, or it is at least in theory as you can see in <a href='https://es.wikipedia.org/wiki/Factorial' target='_blank'>Wiki</a>.

Now hands to work.....

# Swig
## C -C++- Files, and swig interface
In C the files will be (this code is the same you can see on Swig documentation):

### header (.h)
```C
unsigned long long int fact(int n);
```

### body (.c)

This definition is a little different from Swig documentation especifically in data type returned and implementation trying to reduce cyclomatic complexity , being able to calculate until factorial number of 20.

```C
#include "example.h"

unsigned long long int fact(int n) {
    if (n == 0) {
        return 1;
    }else if (n > 0){
        return n * fact(n - 1);
    }
    return 0;
}
```

### swig interface (.i)
On this file is very importante the first line, where you put **%module xxxx***, because this name after **%module** command will be the name of your module
```C
%module example

%{
#define SWIG_FILE_WITH_INIT
#include "example.h"
%}

unsigned long long int fact(int n);
```

## Compiling and getting Swig wrap
Now I've gotten the files with functions I want to use from python, but is necessary recall C and C++ are compiled programming languages, on the other hand Python is an interpretable programming language. But you could be wondering why that information is importante. Well, the reason to recall this info regarding this programming languages is because to build a link between them we need to build -compile- first the C or C++ code, but don't worry, Swig will do this for us, you need to have just the files pointed out above and executing the following command from CMD *-if you are using windows-* or Terminal *-if you are using Linux or Mac-*.

```bash
swig -python example.i
```

Don't forget I've coded a file named *example.i*, for that reason if you're going to change this file name, you must change it in the command line too. This command will build some additional files in your folder, more especifically one named **example_wrap.c**. It is very important point out always a file like this will be created highligthing this file will have a name related to your **.i** filename, just adding **_wrap.c** at the end.

## Creating a Python module

Having this file you are already prepared for building your python module. For making the python module, python has a very useful module named distutils which you can built extensions and wheels in, and some other kind of packages for python modularity. This module is as easy to use as just create the next python file and execute a command line, thus:

```python
#!/usr/bin/env python

"""
setup.py file for SWIG example
"""

from distutils.core import setup, Extension

example_module = Extension('_example', sources=['example_wrap.c','example.c'])

setup(
    name = 'example',
    version = '0.1',
    author = 'Your Name',
    ext_modules=[example_module],
    py_modules=['example']
)
```

If you remember just some lines above I pointed out the swig command will create a file named in this example *example_wrap.c*. This file will be an important piece because it will have C compiler directives to traduce some functionalities between C -or C++- and python, and that is the reason we need to compile the C(++) files at first and then we build the python module.

After the previous step, we can build the extension using again a CMD or Terminal, depending what OS you are using, like this, taking into account this command line must generate a new file with **.pyd** extension who will be a dynamic library used to python portability programs:

```bash
python setup.py build_ext --inplace
```

And that's all to have a module name *example*. So, we can test it in performance, measured by execution time.

# Performance Test

To test it, it makes sense to build exactly the same algorithm for calculating factorial numbers from python code, as was done in C -or C++- programming language.

I'll use the magic command **%%timeit** of jupyter notebook

## Python code

In [1]:
def factorial_py(n):
    if n == 0:
        return 1
    elif n > 0:
        return n * factorial_py(n-1)
    return 0

## Importing python module, built in C

In [2]:
import example

## Comparing time performance

Taking a look on results I got as shown below it is necessary take care on value type returned from C++ function because if you exceed the boundaries in bites on it you'll get a wrong result from C++ funtion.

In [3]:
print('Python result \t:',factorial_py(20))
print('C++ result \t:',example.fact(20))

Python result 	: 2432902008176640000
C++ result 	: 2432902008176640000


### Python

In [4]:
%%timeit -n 10000
factorial_py(20)

2.52 µs ± 209 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


### C or C++ using swig

In [5]:
%%timeit -n 10000
example.fact(20)

203 ns ± 13.6 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


Those implementations that require calling a class and its methods, refer to the example provided in [here](https://web.mit.edu/svn/src/swig-1.3.25/Examples/python/class/)

# Conclusion

At first glance when you try to execute many times both python code and the module built in C, you can calculate the average of time performance for each one, and the most of the times you'll get that the code created with C programming language through swig framework has better performance, measured in lower time of execution. With this in mind I'm tempted to say that buildind some calculations in C or C++ and passing it to python for integrating it into a project could be much better than build it on pure python programming language.

It is important to hightlight you must be careful when you are coding C++ functions which could return numbers greater than supported by each data type, for example in factorial function of numbers greater than 20, the result can't be stored within a **unsigned long long int** variable, this is a very kwnown trade off between python and C or C++, being python very easy to code without take care of this kind of issues.

<hr>

This could not to be the final version, so if you have some recommendation or comment about this, I will be grateful to hear it, using e-mail <b>craquinterogo@unal.edu.co</b> or <b>cristian.quintero@est.uexternado.edu.co</b>